### Q-Learning Approach

* let Q(S,M) be a function that maps a game state S
for board and an action M for move to a value
* Each time the learner sees that game S leads to
game state S' when move M is taken, uses its estimate
of the value of the best move to take from S' to update
Q(S,M)

In [5]:
# BLACK JACK IMPLEMENTATION
import gym
import graphviz
from sklearn import tree
import random

In [6]:
def actionpicker(clf, env, observation, epsilon):
  if clf == 0:
    action = env.action_space.sample()
  else:
    pred = clf.predict([observation + (0,), observation + (1,)])
    action = 1*(pred[1]>pred[0])
  if random.random() < epsilon:
    action = env.action_space.sample()
  return(action)

In [7]:
def train(dat, lab):
  clf = tree.DecisionTreeRegressor(max_leaf_nodes = 6)
  clf = clf.fit(dat, lab)
  return(clf)

In [9]:
env = gym.make("Blackjack-v0")
epochs = 5
N = 100000
epsilon = 0.1
clf = 0

for epoch in range(epochs):
  dat = []
  lab = []
  wins = 0
  for _ in range(N):
    done = False
    observation = env.reset()
    while not done:
      # pick an action
      action = actionpicker(clf, env, observation, epsilon)

      dat += [observation + (action,)]
      observation, reward, done, info = env.step(action)
      if done:
        target = reward
      elif epoch == 0:
        target = 0
      else:
        pred = clf.predict([observation + (0,), observation + (1,)])
        target = max(pred)
      lab += [target]
      if reward > 0.0: wins += 1
  clf = train(dat, lab)

  dot_data = tree.export_graphviz(clf, # class_names = [-1, 0, 1],
                                feature_names = ["holding", "dealer", "ace", "action"], filled=True, rounded=True)
  graph = graphviz.Source(dot_data)
  env.close()
#  print(dat)
  print(wins)
graph

28064
40924
37524
37675
37452


ExecutableNotFound: failed to execute ['dot', '-Tsvg'], make sure the Graphviz executables are on your systems' PATH